# End Game
## The data splits were pre-made(for Ensembling purposes)
## Preprocessing has been done.

In [65]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [17]:
X_train = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\X_train")
X_test = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\X_test")
y_train = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\y_train", header=None)
y_test = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\y_test", header=None)

In [18]:
X_train.head()

,Unnamed: 0,index,user,item,review,rating,date
0,403839,403839,202383,3421,i fell madly in love with these tiny sandwich ...,5.0,2013-12-01
1,100777,100777,74648,1809,my review will strictly be on what i order let...,3.0,2013-09-06
2,512766,512766,237090,4281,im from ny and i can say this pizza be delicio...,2.0,2012-12-20
3,502763,502763,16029,4160,i have never have coffee i enjoy in fact i thi...,5.0,2013-10-14
4,445912,445912,8572,4840,the mediterranean version of chipotle decor be...,4.0,2013-06-21


In [19]:
y_train.head()
#1 for good
#0 for bad
def binarize(n):
    if n==-1:
        return 1
    else:
        return 0

y_train[1] = y_train[1].apply(lambda s: binarize(s))
y_test[1] = y_test[1].apply(lambda s: binarize(s))
X_train['review'] = X_train['review'].apply(lambda s: str(s))
X_test['review'] = X_test['review'].apply(lambda s: str(s))

### Vectorizing data using Tf-Idf Vectorizer with Unigrams and max features of 40,000

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=40000,ngram_range=(1,1))
tvec1.fit(X_train['review'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=40000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [22]:
x_train_tfidf = tvec1.transform(X_train['review'])
x_validation_tfidf = tvec1.transform(X_test['review'])

### Testing out a LogisticRegression model to get the gist
#### LogisticRegression is VERY FAST!

In [24]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train_tfidf, y_train[1])

C:\Users\elonm\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 23.1 s


In [66]:
y_pred = clf.predict(x_validation_tfidf)
print(classification_report(y_pred, y_test[1]))

              precision    recall  f1-score   support

           0       0.99      0.87      0.93    120211
           1       0.05      0.51      0.09      1481

   micro avg       0.87      0.87      0.87    121692
   macro avg       0.52      0.69      0.51    121692
weighted avg       0.98      0.87      0.92    121692



In [27]:
x_train_tfidf.shape

(486766, 40000)

In [100]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [71]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=40000))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 256)               10240256  
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 129       
Total params: 10,273,281
Trainable params: 10,273,281
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train[1], 1000),
                    epochs=2, validation_data=(x_validation_tfidf, y_test[1]),
                    steps_per_epoch=x_train_tfidf.shape[0]/1000, verbose=1)

Epoch 1/2


204/486 [===========>..................] - ETA: 12:28 - loss: 0.6936 - acc: 0.44 - ETA: 8:35 - loss: 0.6639 - acc: 0.6545 - ETA: 7:22 - loss: 0.6241 - acc: 0.727 - ETA: 6:47 - loss: 0.5866 - acc: 0.762 - ETA: 6:26 - loss: 0.5531 - acc: 0.785 - ETA: 6:21 - loss: 0.5250 - acc: 0.800 - ETA: 6:14 - loss: 0.5055 - acc: 0.809 - ETA: 6:04 - loss: 0.4911 - acc: 0.816 - ETA: 5:56 - loss: 0.4828 - acc: 0.820 - ETA: 5:48 - loss: 0.4729 - acc: 0.825 - ETA: 5:43 - loss: 0.4595 - acc: 0.832 - ETA: 5:37 - loss: 0.4527 - acc: 0.835 - ETA: 5:33 - loss: 0.4470 - acc: 0.837 - ETA: 5:30 - loss: 0.4416 - acc: 0.840 - ETA: 5:29 - loss: 0.4367 - acc: 0.842 - ETA: 5:28 - loss: 0.4339 - acc: 0.843 - ETA: 5:28 - loss: 0.4306 - acc: 0.844 - ETA: 5:32 - loss: 0.4283 - acc: 0.845 - ETA: 5:31 - loss: 0.4246 - acc: 0.847 - ETA: 5:31 - loss: 0.4236 - acc: 0.847 - ETA: 5:31 - loss: 0.4217 - acc: 0.848 - ETA: 5:31 - loss: 0.4187 - acc: 0.849 - ETA: 5:28 - loss: 0.4168 - acc: 0.850 - ETA: 5:24 - loss: 0.4145 - acc: 0.85

409/486 [========================>.....] - ETA: 2:52 - loss: 0.3609 - acc: 0.866 - ETA: 2:52 - loss: 0.3608 - acc: 0.866 - ETA: 2:51 - loss: 0.3610 - acc: 0.866 - ETA: 2:50 - loss: 0.3609 - acc: 0.866 - ETA: 2:50 - loss: 0.3607 - acc: 0.866 - ETA: 2:49 - loss: 0.3607 - acc: 0.866 - ETA: 2:48 - loss: 0.3605 - acc: 0.866 - ETA: 2:48 - loss: 0.3605 - acc: 0.866 - ETA: 2:47 - loss: 0.3605 - acc: 0.866 - ETA: 2:46 - loss: 0.3603 - acc: 0.866 - ETA: 2:46 - loss: 0.3603 - acc: 0.866 - ETA: 2:45 - loss: 0.3602 - acc: 0.866 - ETA: 2:44 - loss: 0.3601 - acc: 0.866 - ETA: 2:44 - loss: 0.3600 - acc: 0.866 - ETA: 2:43 - loss: 0.3599 - acc: 0.866 - ETA: 2:42 - loss: 0.3599 - acc: 0.866 - ETA: 2:42 - loss: 0.3599 - acc: 0.866 - ETA: 2:41 - loss: 0.3600 - acc: 0.866 - ETA: 2:40 - loss: 0.3600 - acc: 0.866 - ETA: 2:40 - loss: 0.3600 - acc: 0.866 - ETA: 2:39 - loss: 0.3599 - acc: 0.866 - ETA: 2:38 - loss: 0.3599 - acc: 0.866 - ETA: 2:38 - loss: 0.3597 - acc: 0.866 - ETA: 2:37 - loss: 0.3596 - acc: 0.866

487/486 [==============================] - ETA: 45s - loss: 0.3562 - acc: 0.86 - ETA: 45s - loss: 0.3562 - acc: 0.86 - ETA: 44s - loss: 0.3562 - acc: 0.86 - ETA: 43s - loss: 0.3563 - acc: 0.86 - ETA: 43s - loss: 0.3563 - acc: 0.86 - ETA: 42s - loss: 0.3563 - acc: 0.86 - ETA: 42s - loss: 0.3563 - acc: 0.86 - ETA: 41s - loss: 0.3562 - acc: 0.86 - ETA: 40s - loss: 0.3562 - acc: 0.86 - ETA: 40s - loss: 0.3563 - acc: 0.86 - ETA: 39s - loss: 0.3563 - acc: 0.86 - ETA: 39s - loss: 0.3563 - acc: 0.86 - ETA: 38s - loss: 0.3563 - acc: 0.86 - ETA: 37s - loss: 0.3564 - acc: 0.86 - ETA: 37s - loss: 0.3564 - acc: 0.86 - ETA: 36s - loss: 0.3564 - acc: 0.86 - ETA: 36s - loss: 0.3564 - acc: 0.86 - ETA: 35s - loss: 0.3564 - acc: 0.86 - ETA: 34s - loss: 0.3563 - acc: 0.86 - ETA: 34s - loss: 0.3563 - acc: 0.86 - ETA: 33s - loss: 0.3563 - acc: 0.86 - ETA: 33s - loss: 0.3563 - acc: 0.86 - ETA: 32s - loss: 0.3563 - acc: 0.86 - ETA: 31s - loss: 0.3562 - acc: 0.86 - ETA: 31s - loss: 0.3562 - acc: 0.86 - ETA: 30

204/486 [===========>..................] - ETA: 4:47 - loss: 0.3371 - acc: 0.867 - ETA: 4:48 - loss: 0.3463 - acc: 0.863 - ETA: 4:48 - loss: 0.3415 - acc: 0.870 - ETA: 4:46 - loss: 0.3471 - acc: 0.866 - ETA: 4:46 - loss: 0.3459 - acc: 0.867 - ETA: 4:45 - loss: 0.3457 - acc: 0.867 - ETA: 4:44 - loss: 0.3443 - acc: 0.867 - ETA: 4:47 - loss: 0.3477 - acc: 0.865 - ETA: 4:48 - loss: 0.3485 - acc: 0.864 - ETA: 4:51 - loss: 0.3477 - acc: 0.866 - ETA: 4:53 - loss: 0.3452 - acc: 0.867 - ETA: 4:54 - loss: 0.3454 - acc: 0.866 - ETA: 4:55 - loss: 0.3473 - acc: 0.865 - ETA: 4:57 - loss: 0.3462 - acc: 0.866 - ETA: 5:00 - loss: 0.3482 - acc: 0.866 - ETA: 5:03 - loss: 0.3459 - acc: 0.867 - ETA: 5:06 - loss: 0.3442 - acc: 0.868 - ETA: 5:04 - loss: 0.3435 - acc: 0.868 - ETA: 5:02 - loss: 0.3440 - acc: 0.868 - ETA: 5:02 - loss: 0.3438 - acc: 0.868 - ETA: 5:01 - loss: 0.3427 - acc: 0.869 - ETA: 5:02 - loss: 0.3429 - acc: 0.869 - ETA: 5:01 - loss: 0.3439 - acc: 0.868 - ETA: 5:02 - loss: 0.3435 - acc: 0.868

409/486 [========================>.....] - ETA: 2:52 - loss: 0.3440 - acc: 0.868 - ETA: 2:52 - loss: 0.3439 - acc: 0.868 - ETA: 2:51 - loss: 0.3439 - acc: 0.868 - ETA: 2:50 - loss: 0.3438 - acc: 0.868 - ETA: 2:50 - loss: 0.3438 - acc: 0.868 - ETA: 2:49 - loss: 0.3439 - acc: 0.868 - ETA: 2:48 - loss: 0.3439 - acc: 0.868 - ETA: 2:48 - loss: 0.3440 - acc: 0.868 - ETA: 2:47 - loss: 0.3439 - acc: 0.868 - ETA: 2:46 - loss: 0.3441 - acc: 0.868 - ETA: 2:46 - loss: 0.3441 - acc: 0.868 - ETA: 2:45 - loss: 0.3441 - acc: 0.868 - ETA: 2:44 - loss: 0.3441 - acc: 0.868 - ETA: 2:44 - loss: 0.3441 - acc: 0.868 - ETA: 2:43 - loss: 0.3443 - acc: 0.868 - ETA: 2:43 - loss: 0.3442 - acc: 0.868 - ETA: 2:42 - loss: 0.3441 - acc: 0.868 - ETA: 2:41 - loss: 0.3440 - acc: 0.868 - ETA: 2:41 - loss: 0.3440 - acc: 0.868 - ETA: 2:40 - loss: 0.3440 - acc: 0.868 - ETA: 2:39 - loss: 0.3440 - acc: 0.868 - ETA: 2:39 - loss: 0.3439 - acc: 0.868 - ETA: 2:38 - loss: 0.3439 - acc: 0.868 - ETA: 2:37 - loss: 0.3439 - acc: 0.868

487/486 [==============================] - ETA: 46s - loss: 0.3453 - acc: 0.86 - ETA: 46s - loss: 0.3453 - acc: 0.86 - ETA: 45s - loss: 0.3455 - acc: 0.86 - ETA: 45s - loss: 0.3455 - acc: 0.86 - ETA: 44s - loss: 0.3455 - acc: 0.86 - ETA: 43s - loss: 0.3455 - acc: 0.86 - ETA: 43s - loss: 0.3455 - acc: 0.86 - ETA: 42s - loss: 0.3455 - acc: 0.86 - ETA: 41s - loss: 0.3456 - acc: 0.86 - ETA: 41s - loss: 0.3456 - acc: 0.86 - ETA: 40s - loss: 0.3456 - acc: 0.86 - ETA: 40s - loss: 0.3455 - acc: 0.86 - ETA: 39s - loss: 0.3455 - acc: 0.86 - ETA: 38s - loss: 0.3455 - acc: 0.86 - ETA: 38s - loss: 0.3455 - acc: 0.86 - ETA: 37s - loss: 0.3455 - acc: 0.86 - ETA: 37s - loss: 0.3455 - acc: 0.86 - ETA: 36s - loss: 0.3455 - acc: 0.86 - ETA: 35s - loss: 0.3455 - acc: 0.86 - ETA: 35s - loss: 0.3455 - acc: 0.86 - ETA: 34s - loss: 0.3455 - acc: 0.86 - ETA: 33s - loss: 0.3455 - acc: 0.86 - ETA: 33s - loss: 0.3454 - acc: 0.86 - ETA: 32s - loss: 0.3454 - acc: 0.86 - ETA: 32s - loss: 0.3455 - acc: 0.86 - ETA: 31

In [77]:
y_pred = model.predict(x_validation_tfidf)

In [98]:
res = []

for i in y_pred:
    if i > 0.2:
        res.append(1)
    else:
        res.append(0)

In [99]:
print(classification_report(y_test[1], res))

              precision    recall  f1-score   support

           0       0.92      0.81      0.86    105656
           1       0.29      0.51      0.37     16036

   micro avg       0.77      0.77      0.77    121692
   macro avg       0.60      0.66      0.61    121692
weighted avg       0.83      0.77      0.79    121692



## The main issue (other than a not so good dataset and not so straight problem) - CLASS IMBALANCE 
## There are two ways to tackle this issue: 1. Random Undersampling, 2. SMOTE(Over sampling)

### Over Sampling

In [102]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(x_train_tfidf, y_train[1])

In [108]:
print(X_SMOTE.shape)

(844726, 40000)


In [109]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=40000))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 256)               10240256  
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 129       
Total params: 10,273,281
Trainable params: 10,273,281
Non-trainable params: 0
_________________________________________________________________


In [113]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [114]:
model.fit_generator(generator=batch_generator_shuffle(X_SMOTE, np.asarray(y_SMOTE), 1000),
                    epochs=2, validation_data=(x_validation_tfidf, y_test[1]),
                    steps_per_epoch=x_train_tfidf.shape[0]/1000, verbose=1)

Epoch 1/2


204/486 [===========>..................] - ETA: 13:10 - loss: 0.6907 - acc: 0.84 - ETA: 8:58 - loss: 0.6564 - acc: 0.9245 - ETA: 7:34 - loss: 0.6104 - acc: 0.949 - ETA: 6:51 - loss: 0.5622 - acc: 0.962 - ETA: 6:24 - loss: 0.5138 - acc: 0.969 - ETA: 6:08 - loss: 0.4684 - acc: 0.974 - ETA: 5:55 - loss: 0.4299 - acc: 0.978 - ETA: 5:45 - loss: 0.3953 - acc: 0.981 - ETA: 5:37 - loss: 0.3656 - acc: 0.983 - ETA: 5:31 - loss: 0.3403 - acc: 0.984 - ETA: 5:26 - loss: 0.3174 - acc: 0.986 - ETA: 5:21 - loss: 0.2978 - acc: 0.987 - ETA: 5:17 - loss: 0.2800 - acc: 0.988 - ETA: 5:13 - loss: 0.2648 - acc: 0.989 - ETA: 5:10 - loss: 0.2509 - acc: 0.989 - ETA: 5:07 - loss: 0.2385 - acc: 0.990 - ETA: 5:04 - loss: 0.2269 - acc: 0.991 - ETA: 5:02 - loss: 0.2165 - acc: 0.991 - ETA: 5:00 - loss: 0.2069 - acc: 0.992 - ETA: 4:58 - loss: 0.1982 - acc: 0.992 - ETA: 4:57 - loss: 0.1903 - acc: 0.992 - ETA: 4:55 - loss: 0.1828 - acc: 0.993 - ETA: 4:53 - loss: 0.1759 - acc: 0.993 - ETA: 4:52 - loss: 0.1695 - acc: 0.99

409/486 [========================>.....] - ETA: 3:09 - loss: 0.0213 - acc: 0.999 - ETA: 3:09 - loss: 0.0212 - acc: 0.999 - ETA: 3:08 - loss: 0.0211 - acc: 0.999 - ETA: 3:07 - loss: 0.0210 - acc: 0.999 - ETA: 3:06 - loss: 0.0209 - acc: 0.999 - ETA: 3:06 - loss: 0.0208 - acc: 0.999 - ETA: 3:05 - loss: 0.0207 - acc: 0.999 - ETA: 3:04 - loss: 0.0206 - acc: 0.999 - ETA: 3:03 - loss: 0.0205 - acc: 0.999 - ETA: 3:03 - loss: 0.0204 - acc: 0.999 - ETA: 3:02 - loss: 0.0203 - acc: 0.999 - ETA: 3:01 - loss: 0.0202 - acc: 0.999 - ETA: 3:01 - loss: 0.0201 - acc: 0.999 - ETA: 3:00 - loss: 0.0200 - acc: 0.999 - ETA: 2:59 - loss: 0.0199 - acc: 0.999 - ETA: 2:58 - loss: 0.0198 - acc: 0.999 - ETA: 2:58 - loss: 0.0197 - acc: 0.999 - ETA: 2:57 - loss: 0.0197 - acc: 0.999 - ETA: 2:56 - loss: 0.0196 - acc: 0.999 - ETA: 2:55 - loss: 0.0195 - acc: 0.999 - ETA: 2:55 - loss: 0.0194 - acc: 0.999 - ETA: 2:54 - loss: 0.0193 - acc: 0.999 - ETA: 2:53 - loss: 0.0192 - acc: 0.999 - ETA: 2:52 - loss: 0.0191 - acc: 0.999

487/486 [==============================] - ETA: 48s - loss: 0.0106 - acc: 0.99 - ETA: 48s - loss: 0.0106 - acc: 0.99 - ETA: 47s - loss: 0.0106 - acc: 0.99 - ETA: 47s - loss: 0.0106 - acc: 0.99 - ETA: 46s - loss: 0.0105 - acc: 0.99 - ETA: 45s - loss: 0.0105 - acc: 0.99 - ETA: 45s - loss: 0.0105 - acc: 0.99 - ETA: 44s - loss: 0.0105 - acc: 0.99 - ETA: 43s - loss: 0.0104 - acc: 0.99 - ETA: 43s - loss: 0.0104 - acc: 0.99 - ETA: 42s - loss: 0.0104 - acc: 0.99 - ETA: 41s - loss: 0.0104 - acc: 0.99 - ETA: 41s - loss: 0.0103 - acc: 0.99 - ETA: 40s - loss: 0.0103 - acc: 0.99 - ETA: 39s - loss: 0.0103 - acc: 0.99 - ETA: 39s - loss: 0.0103 - acc: 0.99 - ETA: 38s - loss: 0.0102 - acc: 0.99 - ETA: 38s - loss: 0.0102 - acc: 0.99 - ETA: 37s - loss: 0.0102 - acc: 0.99 - ETA: 36s - loss: 0.0102 - acc: 0.99 - ETA: 36s - loss: 0.0102 - acc: 0.99 - ETA: 35s - loss: 0.0101 - acc: 0.99 - ETA: 34s - loss: 0.0101 - acc: 0.99 - ETA: 34s - loss: 0.0101 - acc: 0.99 - ETA: 33s - loss: 0.0101 - acc: 0.99 - ETA: 32

186/486 [==========>...................] - ETA: 5:03 - loss: 1.0179e-07 - acc: 1.000 - ETA: 5:10 - loss: 1.0179e-07 - acc: 1.000 - ETA: 5:00 - loss: 1.0134e-07 - acc: 1.000 - ETA: 4:57 - loss: 1.0107e-07 - acc: 1.000 - ETA: 4:53 - loss: 1.0088e-07 - acc: 1.000 - ETA: 4:50 - loss: 1.0115e-07 - acc: 1.000 - ETA: 4:48 - loss: 1.0129e-07 - acc: 1.000 - ETA: 4:46 - loss: 1.0117e-07 - acc: 1.000 - ETA: 4:46 - loss: 1.0122e-07 - acc: 1.000 - ETA: 4:44 - loss: 1.0118e-07 - acc: 1.000 - ETA: 4:42 - loss: 1.0108e-07 - acc: 1.000 - ETA: 4:47 - loss: 1.0106e-07 - acc: 1.000 - ETA: 4:51 - loss: 1.0116e-07 - acc: 1.000 - ETA: 4:51 - loss: 1.0109e-07 - acc: 1.000 - ETA: 4:50 - loss: 1.0110e-07 - acc: 1.000 - ETA: 4:50 - loss: 1.0108e-07 - acc: 1.000 - ETA: 4:48 - loss: 1.0103e-07 - acc: 1.000 - ETA: 4:46 - loss: 1.0101e-07 - acc: 1.000 - ETA: 4:45 - loss: 1.0114e-07 - acc: 1.000 - ETA: 4:44 - loss: 1.0116e-07 - acc: 1.000 - ETA: 4:43 - loss: 1.0122e-07 - acc: 1.000 - ETA: 4:42 - loss: 1.0121e-07 - ac

372/486 [=====================>........] - ETA: 2:57 - loss: 1.0099e-07 - acc: 1.000 - ETA: 2:56 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:56 - loss: 1.0099e-07 - acc: 1.000 - ETA: 2:55 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:54 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:54 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:53 - loss: 1.0097e-07 - acc: 1.000 - ETA: 2:53 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:52 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:51 - loss: 1.0098e-07 - acc: 1.000 - ETA: 2:51 - loss: 1.0097e-07 - acc: 1.000 - ETA: 2:50 - loss: 1.0097e-07 - acc: 1.000 - ETA: 2:50 - loss: 1.0097e-07 - acc: 1.000 - ETA: 2:49 - loss: 1.0097e-07 - acc: 1.000 - ETA: 2:48 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:48 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:47 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:47 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:46 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:45 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:45 - loss: 1.0096e-07 - acc: 1.000 - ETA: 2:44 - loss: 1.0096e-07 - ac

487/486 [==============================] - ETA: 1:07 - loss: 1.0076e-07 - acc: 1.000 - ETA: 1:06 - loss: 1.0076e-07 - acc: 1.000 - ETA: 1:06 - loss: 1.0076e-07 - acc: 1.000 - ETA: 1:05 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:05 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:04 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:04 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:03 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:02 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:02 - loss: 1.0075e-07 - acc: 1.000 - ETA: 1:01 - loss: 1.0074e-07 - acc: 1.000 - ETA: 1:01 - loss: 1.0074e-07 - acc: 1.000 - ETA: 1:00 - loss: 1.0074e-07 - acc: 1.000 - ETA: 59s - loss: 1.0074e-07 - acc: 1.000 - ETA: 59s - loss: 1.0074e-07 - acc: 1.00 - ETA: 58s - loss: 1.0074e-07 - acc: 1.00 - ETA: 58s - loss: 1.0074e-07 - acc: 1.00 - ETA: 57s - loss: 1.0074e-07 - acc: 1.00 - ETA: 56s - loss: 1.0074e-07 - acc: 1.00 - ETA: 56s - loss: 1.0074e-07 - acc: 1.00 - ETA: 55s - loss: 1.0073e-07 - acc: 1.00 - ETA: 55s - loss: 1.0073e-07 - acc: 1.00 - ETA: 5

In [118]:
y_pred = model.predict(x_validation_tfidf)

C:\Users\elonm\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.87      1.00      0.93    105656
           1       0.00      0.00      0.00     16036

   micro avg       0.87      0.87      0.87    121692
   macro avg       0.43      0.50      0.46    121692
weighted avg       0.75      0.87      0.81    121692



In [124]:
res = []
for i in y_pred:
    if i > 0.4:
        res.append(1)
    else:
        res.append(0)
        
print(classification_report(y_test[1], res))

C:\Users\elonm\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.87      1.00      0.93    105656
           1       0.00      0.00      0.00     16036

   micro avg       0.87      0.87      0.87    121692
   macro avg       0.43      0.50      0.46    121692
weighted avg       0.75      0.87      0.81    121692



## Under Sampling

In [125]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(x_train_tfidf, y_train[1])

In [126]:
print(X_rus.shape)

(128806, 40000)


In [136]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=40000))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 128)               5120128   
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 129       
Total params: 5,136,769
Trainable params: 5,136,769
Non-trainable params: 0
_________________________________________________________________


In [137]:
model.fit_generator(generator=batch_generator_shuffle(X_rus, np.asarray(y_rus), 1000),
                    epochs=2, validation_data=(x_validation_tfidf, y_test[1]),
                    steps_per_epoch=x_train_tfidf.shape[0]/1000, verbose=1)

Epoch 1/2


204/486 [===========>..................] - ETA: 11:13 - loss: 0.6919 - acc: 0.67 - ETA: 7:22 - loss: 0.6644 - acc: 0.8375 - ETA: 6:02 - loss: 0.6310 - acc: 0.891 - ETA: 5:21 - loss: 0.5942 - acc: 0.918 - ETA: 4:56 - loss: 0.5582 - acc: 0.935 - ETA: 4:39 - loss: 0.5230 - acc: 0.945 - ETA: 4:27 - loss: 0.4886 - acc: 0.953 - ETA: 4:18 - loss: 0.4567 - acc: 0.959 - ETA: 4:11 - loss: 0.4281 - acc: 0.963 - ETA: 4:06 - loss: 0.4016 - acc: 0.967 - ETA: 4:01 - loss: 0.3777 - acc: 0.970 - ETA: 3:57 - loss: 0.3560 - acc: 0.972 - ETA: 3:54 - loss: 0.3364 - acc: 0.975 - ETA: 3:51 - loss: 0.3187 - acc: 0.976 - ETA: 3:48 - loss: 0.3026 - acc: 0.978 - ETA: 3:46 - loss: 0.2877 - acc: 0.979 - ETA: 3:44 - loss: 0.2742 - acc: 0.980 - ETA: 3:42 - loss: 0.2617 - acc: 0.981 - ETA: 3:40 - loss: 0.2503 - acc: 0.982 - ETA: 3:38 - loss: 0.2399 - acc: 0.983 - ETA: 3:36 - loss: 0.2302 - acc: 0.984 - ETA: 3:35 - loss: 0.2212 - acc: 0.985 - ETA: 3:34 - loss: 0.2129 - acc: 0.985 - ETA: 3:32 - loss: 0.2052 - acc: 0.98

411/486 [========================>.....] - ETA: 2:05 - loss: 0.0256 - acc: 0.998 - ETA: 2:05 - loss: 0.0255 - acc: 0.998 - ETA: 2:04 - loss: 0.0254 - acc: 0.998 - ETA: 2:04 - loss: 0.0253 - acc: 0.998 - ETA: 2:03 - loss: 0.0251 - acc: 0.998 - ETA: 2:03 - loss: 0.0250 - acc: 0.998 - ETA: 2:03 - loss: 0.0249 - acc: 0.998 - ETA: 2:02 - loss: 0.0248 - acc: 0.998 - ETA: 2:02 - loss: 0.0247 - acc: 0.998 - ETA: 2:01 - loss: 0.0245 - acc: 0.998 - ETA: 2:01 - loss: 0.0244 - acc: 0.998 - ETA: 2:00 - loss: 0.0243 - acc: 0.998 - ETA: 2:00 - loss: 0.0242 - acc: 0.998 - ETA: 1:59 - loss: 0.0241 - acc: 0.998 - ETA: 1:59 - loss: 0.0240 - acc: 0.998 - ETA: 1:59 - loss: 0.0239 - acc: 0.998 - ETA: 1:58 - loss: 0.0238 - acc: 0.998 - ETA: 1:58 - loss: 0.0237 - acc: 0.998 - ETA: 1:57 - loss: 0.0236 - acc: 0.998 - ETA: 1:57 - loss: 0.0234 - acc: 0.998 - ETA: 1:56 - loss: 0.0233 - acc: 0.998 - ETA: 1:56 - loss: 0.0232 - acc: 0.998 - ETA: 1:55 - loss: 0.0231 - acc: 0.998 - ETA: 1:55 - loss: 0.0230 - acc: 0.998

487/486 [==============================] - ETA: 33s - loss: 0.0127 - acc: 0.99 - ETA: 33s - loss: 0.0127 - acc: 0.99 - ETA: 32s - loss: 0.0127 - acc: 0.99 - ETA: 32s - loss: 0.0127 - acc: 0.99 - ETA: 31s - loss: 0.0126 - acc: 0.99 - ETA: 31s - loss: 0.0126 - acc: 0.99 - ETA: 30s - loss: 0.0126 - acc: 0.99 - ETA: 30s - loss: 0.0125 - acc: 0.99 - ETA: 29s - loss: 0.0125 - acc: 0.99 - ETA: 29s - loss: 0.0125 - acc: 0.99 - ETA: 28s - loss: 0.0124 - acc: 0.99 - ETA: 28s - loss: 0.0124 - acc: 0.99 - ETA: 28s - loss: 0.0124 - acc: 0.99 - ETA: 27s - loss: 0.0124 - acc: 0.99 - ETA: 27s - loss: 0.0123 - acc: 0.99 - ETA: 26s - loss: 0.0123 - acc: 0.99 - ETA: 26s - loss: 0.0123 - acc: 0.99 - ETA: 25s - loss: 0.0122 - acc: 0.99 - ETA: 25s - loss: 0.0122 - acc: 0.99 - ETA: 24s - loss: 0.0122 - acc: 0.99 - ETA: 24s - loss: 0.0122 - acc: 0.99 - ETA: 24s - loss: 0.0121 - acc: 0.99 - ETA: 23s - loss: 0.0121 - acc: 0.99 - ETA: 23s - loss: 0.0121 - acc: 0.99 - ETA: 22s - loss: 0.0120 - acc: 0.99 - ETA: 22

186/486 [==========>...................] - ETA: 4:17 - loss: 1.0017e-07 - acc: 1.000 - ETA: 4:29 - loss: 1.0009e-07 - acc: 1.000 - ETA: 4:13 - loss: 1.0903e-07 - acc: 1.000 - ETA: 4:08 - loss: 1.0687e-07 - acc: 1.000 - ETA: 4:09 - loss: 1.0757e-07 - acc: 1.000 - ETA: 4:04 - loss: 1.0660e-07 - acc: 1.000 - ETA: 4:00 - loss: 1.0610e-07 - acc: 1.000 - ETA: 4:00 - loss: 1.0594e-07 - acc: 1.000 - ETA: 3:57 - loss: 1.0547e-07 - acc: 1.000 - ETA: 4:01 - loss: 1.0498e-07 - acc: 1.000 - ETA: 3:59 - loss: 1.0502e-07 - acc: 1.000 - ETA: 3:59 - loss: 1.0496e-07 - acc: 1.000 - ETA: 4:02 - loss: 1.0502e-07 - acc: 1.000 - ETA: 4:04 - loss: 1.0468e-07 - acc: 1.000 - ETA: 4:07 - loss: 1.0437e-07 - acc: 1.000 - ETA: 4:05 - loss: 1.0414e-07 - acc: 1.000 - ETA: 4:07 - loss: 1.0391e-07 - acc: 1.000 - ETA: 4:05 - loss: 1.0434e-07 - acc: 1.000 - ETA: 4:04 - loss: 1.0424e-07 - acc: 1.000 - ETA: 4:02 - loss: 1.0404e-07 - acc: 1.000 - ETA: 3:59 - loss: 1.0406e-07 - acc: 1.000 - ETA: 3:58 - loss: 1.0389e-07 - ac

372/486 [=====================>........] - ETA: 2:17 - loss: 1.0269e-07 - acc: 1.000 - ETA: 2:16 - loss: 1.0267e-07 - acc: 1.000 - ETA: 2:16 - loss: 1.0266e-07 - acc: 1.000 - ETA: 2:15 - loss: 1.0265e-07 - acc: 1.000 - ETA: 2:15 - loss: 1.0265e-07 - acc: 1.000 - ETA: 2:14 - loss: 1.0263e-07 - acc: 1.000 - ETA: 2:14 - loss: 1.0264e-07 - acc: 1.000 - ETA: 2:14 - loss: 1.0262e-07 - acc: 1.000 - ETA: 2:14 - loss: 1.0261e-07 - acc: 1.000 - ETA: 2:13 - loss: 1.0261e-07 - acc: 1.000 - ETA: 2:13 - loss: 1.0260e-07 - acc: 1.000 - ETA: 2:13 - loss: 1.0259e-07 - acc: 1.000 - ETA: 2:12 - loss: 1.0257e-07 - acc: 1.000 - ETA: 2:12 - loss: 1.0256e-07 - acc: 1.000 - ETA: 2:12 - loss: 1.0255e-07 - acc: 1.000 - ETA: 2:12 - loss: 1.0255e-07 - acc: 1.000 - ETA: 2:11 - loss: 1.0253e-07 - acc: 1.000 - ETA: 2:11 - loss: 1.0252e-07 - acc: 1.000 - ETA: 2:10 - loss: 1.0251e-07 - acc: 1.000 - ETA: 2:10 - loss: 1.0250e-07 - acc: 1.000 - ETA: 2:10 - loss: 1.0249e-07 - acc: 1.000 - ETA: 2:10 - loss: 1.0248e-07 - ac

487/486 [==============================] - ETA: 55s - loss: 1.0188e-07 - acc: 1.00 - ETA: 54s - loss: 1.0188e-07 - acc: 1.00 - ETA: 54s - loss: 1.0188e-07 - acc: 1.00 - ETA: 53s - loss: 1.0187e-07 - acc: 1.00 - ETA: 53s - loss: 1.0187e-07 - acc: 1.00 - ETA: 53s - loss: 1.0186e-07 - acc: 1.00 - ETA: 52s - loss: 1.0186e-07 - acc: 1.00 - ETA: 52s - loss: 1.0185e-07 - acc: 1.00 - ETA: 51s - loss: 1.0185e-07 - acc: 1.00 - ETA: 51s - loss: 1.0185e-07 - acc: 1.00 - ETA: 50s - loss: 1.0185e-07 - acc: 1.00 - ETA: 50s - loss: 1.0184e-07 - acc: 1.00 - ETA: 49s - loss: 1.0184e-07 - acc: 1.00 - ETA: 49s - loss: 1.0183e-07 - acc: 1.00 - ETA: 48s - loss: 1.0183e-07 - acc: 1.00 - ETA: 48s - loss: 1.0183e-07 - acc: 1.00 - ETA: 47s - loss: 1.0183e-07 - acc: 1.00 - ETA: 47s - loss: 1.0182e-07 - acc: 1.00 - ETA: 46s - loss: 1.0182e-07 - acc: 1.00 - ETA: 46s - loss: 1.0182e-07 - acc: 1.00 - ETA: 45s - loss: 1.0182e-07 - acc: 1.00 - ETA: 45s - loss: 1.0181e-07 - acc: 1.00 - ETA: 44s - loss: 1.0181e-07 - acc

In [129]:
y_pred = model.predict(x_validation_tfidf)

In [135]:
res = []
for i in y_pred:
    if i > 0.3:
        res.append(1)
    else:
        res.append(0)
        
print(classification_report(y_test[1], res))

C:\Users\elonm\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.87      1.00      0.93    105656
           1       0.00      0.00      0.00     16036

   micro avg       0.87      0.87      0.87    121692
   macro avg       0.43      0.50      0.46    121692
weighted avg       0.75      0.87      0.81    121692



In [138]:
a = 0
b = 0
for i in y_rus:
    if i == 1:
        a += 1
    else:
        b += 1
print(a, b)

64403 64403


# TO HELL WITH THIS!